In [1]:
using DataFrames, CSV, Tables, Plots
using JuMP, Gurobi

In [2]:
const GRB_ENV = Gurobi.Env()

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18


Gurobi.Env(Ptr{Nothing} @0x0000000081880490, false, 0)

In [3]:
network = (CSV.File("./soc-sign-bitcoinotc.csv") |> Tables.matrix)[:, 1:3];

In [4]:
# n = size(network)[1]
n = floor(Int, maximum([maximum(network[:, 1]), maximum(network[:, 2])]))
A = zeros(Int8, n, n)

6005×6005 Matrix{Int8}:
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  

In [5]:
e = size(network)[1]
for i=1:e
    if network[i,3] < 0
        continue
    end
    s = floor(Int, network[i,1])
    d = floor(Int, network[i,2])
    A[s, d] = network[i,3]
end

In [6]:
A = A./10

6005×6005 Matrix{Float64}:
 0.0  0.8  0.6  1.0  0.4  0.8  0.9  0.7  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.8  0.0  0.8  0.5  0.0  0.5  0.5  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.6  0.7  0.0  0.0  0.5  0.7  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.3  0.0  0.0  0.0  0.0  0.3  0.1  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.8  0.0  0.0  0.2  0.2  0.0  0.5  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.7  0.5  0.6  0.4  0.1  0.3  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.3  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.8  0.7  0.7  0.0  0.0  0.7  0.0  0.1     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.8  0.2  0.0  0.8  0.0  0.0  0.1  0

In [7]:
graph = (CSV.File("./graph.csv") |> Tables.matrix);
graph_reach = (CSV.File("./graph_reach.csv") |> Tables.matrix);

In [8]:
synthetic = (CSV.File("./synthetic_1.csv") |> Tables.matrix)
threshold = synthetic[:,6];
budget = synthetic[:,7];

In [14]:
function getInfluence(T, B, output=1)
    model = Model(() -> Gurobi.Optimizer(GRB_ENV));
    set_optimizer_attribute(model, "OutputFlag",  output)
    set_optimizer_attribute(model, "TimeLimit",  60)
    @variable(model, x[1:n,1:T] >= 0, Bin); 
    @constraint(model,  sum(x[:,1].*budget) <= B)
    for t=2:T
        @constraint(model, [i=1:n], sum(x[:,t-1].*A[i,:]) >= threshold[i]*x[i,t])
        @constraint(model, [i=1:n], sum(x[:,t-1].*A[i,:]) <= threshold[i]+x[i,t])
        @constraint(model, [i=1:n], x[i,t-1] <= x[i,t])
    end
    @objective(model, Max, sum(x[:,T]))
    optimize!(model)
    return model
end

getInfluence (generic function with 2 methods)

In [ ]:
T = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
B = [1000000]
#B = [1000, 5000, 10000, 50000, 100000, 500000, 1000000, 5000000, 10000000, 50000000]
results = zeros(size(T)[1], size(B)[1])
for t in T
    for b in B
        model = getInfluence(t, b, 0)
        results[t, 1] = objective_value.(model)
        println("T=$t, B=$b, influence=$(results[t,1])")
    end
end     

Set parameter TimeLimit to value 60
T=1, B=1000000, influence=500.0
Set parameter TimeLimit to value 60
T=2, B=1000000, influence=1767.0
Set parameter TimeLimit to value 60
T=3, B=1000000, influence=2165.0
Set parameter TimeLimit to value 60
T=4, B=1000000, influence=2284.0
Set parameter TimeLimit to value 60
T=5, B=1000000, influence=1828.0

In [16]:
q = value.(x)
for i=1:n
    if q[i][1] == 1
        println("$i, $(budget[i]), $(synthetic[i, 2])")
    end
end

LoadError: UndefVarError: x not defined

In [130]:
for t=1:T
    println("Time Period: $t")
    for i=1:n
        if q[i,t] == 1
            println(i)
        end
    end
end

Time Period: 1
28
97
148
182
231
241
249
319
323
365
388
440
443
444
453
456
510
512
549
557
560
563
564
572
575
578
597
598
627
642
670
671
677
685
690
696
706
715
719
727
728
740
757
776
782
784
788
812
819
839
866
874
879
883
885
893
981
982
993
1020
1054
1062
1067
1078
1086
1088
1090
1096
1097
1147
1149
1164
1165
1176
1186
1187
1200
1202
1210
1212
1216
1231
1249
1250
1255
1280
1312
1341
1349
1350
1362
1365
1427
1432
1433
1465
1504
1513
1514
1579
1590
1669
1703
1898
1906
1908
1931
1968
1969
1992
2042
2133
2202
2208
2211
2244
2268
2352
2363
2401
2410
2443
2444
2471
2618
2735
2736
2757
2765
2766
2813
2814
2836
2888
2939
3060
3065
3148
3151
3210
3275
3313
3328
3369
3602
3622
3762
3763
3885
3960
4019
4035
4043
4111
4123
4126
4158
4232
4329
4366
4368
4390
4406
4413
4432
4433
4495
4496
4563
4564
4630
4658
4714
4746
4749
4750
4814
4818
4832
4847
4859
4869
4965
4966
4968
4980
4981
5037
5039
5059
5060
5186
5192
5195
5197
5198
5215
5345
5359
5367
5390
5421
5428
5429
5448
5479
5483
5553
5556
5